In [84]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad
import scanpy as sc
sns.set_context("poster")
sns.set_palette("colorblind")

In [85]:
adata = ad.read_h5ad("../data/colquitt_raw.h5ad")
adata

AnnData object with n_obs × n_vars = 4028 × 19857
    obs: 'UMAP_1', 'UMAP_2', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'species', 'position2', 'cluster_int_sub2', 'orig.ident', 'cluster_int', 'subclass', 'birth_area', 'Chrna10', 'Chrm2', 'Chrna2', 'Chrm3', 'Chrna9', 'Chrm4', 'Chrm5', 'Chrna1', 'Chrnd', 'Chrng', 'Chrna5', 'Chrna3', 'Chrnb4', 'Chrna4', 'Chrnb2', 'Chrnb3', 'Chrna6', 'Chrna7'

In [86]:
adata.obs['Elfn1'] = np.array(adata[:,"Elfn1"].X)
by_batch = adata.obs.groupby(['subclass']).mean()
sst_expr = by_batch[["Sst" in index for index in by_batch.index]]['Elfn1'].mean()
pvalb_expr = by_batch[["Pvalb" in index for index in by_batch.index]]['Elfn1'].mean()
log2fc = np.log2(sst_expr / pvalb_expr)
print(log2fc)

2.2629976


/tmp/ipykernel_130449/78831764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  by_batch = adata.obs.groupby(['subclass']).mean()


In [87]:
sc.pp.normalize_per_cell(adata, 1e4)
sc.pp.log1p(adata)

In [88]:
sc.tl.rank_genes_groups(adata, "subclass", groups=['Sst', 'Pvalb'])

In [89]:
rank = np.where(adata.uns['rank_genes_groups']['names']['Sst'] == 'Elfn1')[0][0]
print(rank)
adata.uns['rank_genes_groups']['logfoldchanges'] 

3


rec.array([( 2.9014413,  2.9253972), ( 2.14258  ,  3.2953012),
           ( 3.2884436,  4.678059 ), ..., (-2.67052  , -2.7870054),
           (-2.7154448, -3.4419682), (-2.9200366, -3.3581946)],
          dtype=[('Sst', '<f4'), ('Pvalb', '<f4')])

In [90]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
import pandas as pd
df = pd.DataFrame({group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'logfoldchanges']})
logfc = df[df['Sst_n'] == 'Elfn1']['Sst_l']
print(logfc)

3    2.795659
Name: Sst_l, dtype: float32


4